In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components import asset, preference
from tofina.theory import varianceMinimization
import tofina.utils as utils
from tofina.macros import portfolioOptimization, optionPricing
import pandas as pd
import numpy as np
import torch

minVarianceFilePath1 = "minVariancePortfolio1.csv"
covariance1 = torch.tensor([[5, 1], [1, 10]]).float() / 100
mean=0.03


portfolio_ = varianceMinimization.GenerateVarianceMinimizationPortfolio(
    mean, covariance1, torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])
)
optim = portfolioOptimization.optimizeStockPortfolioRiskAverse(
    portfolio_, minVarianceFilePath1
)



../tofina/utils.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  params[key] = torch.nn.Parameter(torch.tensor(val), requires_grad=False)
  9%|▉         | 88/1000 [00:00<00:03, 267.86it/s]


In [3]:
optim.optimizationResult

{'initial_Stock_Company0weight': 0.5,
 'initial_Stock_Company1weight': 0.5,
 'initial_loss': -0.019090916961431503,
 'final_Stock_Company0weight': 0.6950502991676331,
 'final_Stock_Company1weight': 0.30494970083236694,
 'final_loss': -0.020339922979474068,
 'converged': True}

In [1]:
import sys
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
import datetime as dt
import torch
sys.path.append('..')

NameError: name 'torch' is not defined

In [21]:
def prepend_tensor_with_ones(tensor: torch.Tensor) -> torch.Tensor:
    ones = torch.ones((tensor.shape[0],tensor.shape[1], 1))
    return torch.cat((ones, tensor), 2)
SPY = yf.download("SPY")
SPY["diff"] = 100*SPY["Close"].diff() / SPY["Close"].shift(1)
SPY = SPY.dropna()
ts = SPY["diff"]
split_date = dt.datetime(2022, 12, 30)
am = arch_model(ts, vol="Garch", p=3, o=0, q=3, dist="t", mean="AR", lags=5)
res = am.fit(update_freq=5, last_obs=split_date)
simulation = res.forecast(
    method="simulation", horizon=20, simulations=1000
)
simulation_values = (1+simulation.simulations.values/100)
simulation_values = simulation_values.cumprod(axis=2)
simulation_values = torch.from_numpy(simulation_values)
simulation_values = prepend_tensor_with_ones(simulation_values)
init_values = torch.from_numpy(SPY["Close"].values)
date = "2022-12-30"
start_value = SPY["Close"].loc[date]




[*********************100%%**********************]  1 of 1 completed


Iteration:      5,   Func. Count:     94,   Neg. LLF: 30430.821311935004
Iteration:     10,   Func. Count:    181,   Neg. LLF: 14455.181743728463
Iteration:     15,   Func. Count:    263,   Neg. LLF: 10114.572461608102
Iteration:     20,   Func. Count:    343,   Neg. LLF: 10073.527488616097
Iteration:     25,   Func. Count:    420,   Neg. LLF: 10072.070965098497
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10072.07095474302
            Iterations: 27
            Function evaluations: 449
            Gradient evaluations: 27


/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [23]:
init_values.shape

torch.Size([7819])

In [24]:
simulation_values.shape

torch.Size([7818, 1000, 21])